In [111]:
from collections import deque
from typing import Dict, List, Set, Tuple

import numpy as np

In [42]:

def make_board(state):
    """
    Convert a 9-element list into a 3x3 numpy array.
    """
    return np.array(state).reshape(3, 3)

def print_board(board):
    """
    Print a 3x3 numpy array board for the 8-puzzle.
    0 is treated as blank.
    """
    for row in board:
        line = ""
        for val in row:
            if val == 0:
                line += "|    "
            else:
                line += f"| {val:2d} "
        line += "|"
        print(line)
        print("-----+-----+-----")

def board_key(board: np.ndarray):
    """Return a hashable key for a 3x3 numpy board."""
    return board.tobytes()


In [128]:
goal_state = make_board([0,1,2,3,4,5,6,7,8])

print_board(goal_state)

print(board_key(goal_state))

|    |  1 |  2 |
-----+-----+-----
|  3 |  4 |  5 |
-----+-----+-----
|  6 |  7 |  8 |
-----+-----+-----
b'\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00'


In [131]:
def find_valid_moves(board: np.ndarray):
    """
    Finds all valid moves from the 8-tile board and returns those boards as 3x3 numpy arrays.
    Move notation is the direction that the blank space moves.
    """
    r, c = np.where(board == 0)
    out = []
    if c < 2:
        b = board.copy()
        b[r, c], b[r, c+1] = b[r, c+1], b[r, c]
        out.append((b, 'Right'))
    if c > 0:
        b = board.copy()
        b[r, c], b[r, c-1] = b[r, c-1], b[r, c]
        out.append((b, 'Left'))
    if r < 2:
        b = board.copy()
        b[r, c], b[r+1, c] = b[r+1, c], b[r, c]
        out.append((b, 'Down'))
    if r > 0:
        b = board.copy()
        b[r, c], b[r-1, c] = b[r-1, c], b[r, c]
        out.append((b, 'Up'))
    return out


In [124]:
class SearchTree:
    def __init__(self, board):
        self.root = board
        self.children = []
        self.directions = []


    def display(self, depth: int, indent: int = 0):
        """
        Used to display a state tree up to a certain depth.
        Best with small examples when debugging.
        """
        pad = "  " * indent
        print(f"{pad}Node:")
        print_board(self.root)

        if depth <= 0:
            return

        for i, child in enumerate(self.children, 1):
            print(f"{pad}Child {i}:")
            child.display(depth=depth-1, indent=indent+1)

    def find_solution_pruning(self, goal_state, max_depth: int = 1):
        """
        Initial implementation that keeps track of what states have been seen.
        This does not get the best solution because some branches are pruned too early.
        """
        seen: Set[bytes] = {board_key(self.root)}
        stack: List[Tuple[SearchTree, int]] = [(self, 0)]
        goal_state_key = board_key(goal_state)

        if board_key(self.root) == goal_state_key:
            return self.root, 0, []

        while stack:
            node, depth = stack.pop()
            if depth > max_depth:
                continue

            for move_state, direction in find_valid_moves(node.root):
                key = board_key(move_state)
                path = node.directions + [direction]

                if key == goal_state_key:
                    return move_state, depth+1, path

                if key in seen:
                    continue

                seen.add(key)
                new_board = SearchTree(move_state)
                new_board.directions = path
                stack.append((new_board, depth+1))
                node.children.append(new_board)
        return None, None, None

    def find_solution_bfs(self, goal_state):
        """
        Second implementation that uses breadth-first search.
        Finds the optimal solution by iterating over states in order of depth
        """
        goal_state_key = board_key(goal_state)
        start_state_key = board_key(self.root)
        if start_state_key == goal_state_key:
            return self.root, 0, []

        q = deque([(self, 0)])
        best_depth: Dict[bytes, int] = {start_state_key: 0}

        while q:
            node, depth = q.popleft()

            for move_state, direction in find_valid_moves(node.root):
                curr_key = board_key(move_state)
                curr_depth = depth + 1

                # only proceed if new or shallower
                if curr_key in best_depth and curr_depth >= best_depth[curr_key]:
                    continue
                best_depth[curr_key] = curr_depth

                child = SearchTree(move_state)
                child.directions = node.directions + [direction]
                node.children.append(child)

                if curr_key == goal_state_key:
                    return move_state, curr_depth, child.directions

                q.append((child, curr_depth))

        return None, None, None



In [133]:
example_state = make_board([8,7,6,5,4,3,2,1,0])
goal_state = make_board([0,1,2,3,4,5,6,7,8])

solution = SearchTree(example_state)
answer, depth, directions = solution.find_solution_bfs(goal_state)

print("---------- START ----------")
print_board(example_state)
print(f"---------- SOLUTION ({depth} moves) ----------")
print_board(answer)
print(f"---------- DIRECTIONS {directions} ----------")


---------- START ----------
|  8 |  7 |  6 |
-----+-----+-----
|  5 |  4 |  3 |
-----+-----+-----
|  2 |  1 |    |
-----+-----+-----
---------- SOLUTION (28 moves) ----------
|    |  1 |  2 |
-----+-----+-----
|  3 |  4 |  5 |
-----+-----+-----
|  6 |  7 |  8 |
-----+-----+-----
---------- DIRECTIONS ['Left', 'Left', 'Up', 'Up', 'Right', 'Right', 'Down', 'Down', 'Left', 'Left', 'Up', 'Up', 'Right', 'Right', 'Down', 'Down', 'Left', 'Left', 'Up', 'Up', 'Right', 'Right', 'Down', 'Down', 'Left', 'Left', 'Up', 'Up'] ----------
